In [ ]:
from typing import List
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from datetime import datetime
from estival import priors as esp
from estival.sampling import tools as esamp
import pymc as pm
import arviz as az

from emu_renewal.renew import RenewalModel, TruncRenewalModel
from emu_renewal.calibrate import get_wrapped_ll, use_model

In [ ]:
run_in = 30
n_process_periods = 12
raw_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)['MYS']
raw_data.index = pd.to_datetime(raw_data.index)
mys_data = raw_data.loc[datetime(2021, 3, 1): datetime(2021, 11, 1)].reset_index()['MYS']
mys_data.index += run_in
n_times = len(mys_data) + run_in
calib_kwargs = {'pop': 33e6, 'n_times': n_times, 'run_in': run_in, 'targets': mys_data}

In [ ]:
# Define parameter ranges
scalar_req = {
    'gen_mean': {'name': 'Generation time mean (days)', 'init': 5.0, 'lower': 0.1, 'upper': 14.0},
    'gen_sd': {'name': 'Generation time standard deviation (days)', 'init': 5.0, 'lower': 2.5, 'upper': 8.0},
    'cdr': {'name': 'Case detection proportion', 'init': 0.06, 'lower': 0.04, 'upper': 0.2},
    'seed': {'name': 'Peak seed rate', 'init': 1e4, 'lower': 5e3, 'upper': 2e4},
}
params_df = pd.DataFrame(scalar_req).transpose()
proc_req = {'name': 'random_process', 'init': 0.0, 'lower': -2.0, 'upper': 2.0}

In [ ]:
def get_obj_func(model):
    def calib_func_ll(parameters: List[float], pop: int, n_times: int, run_in: int, targets: dict) -> float:
        gen_time_mean, gen_time_sd, cdr, seed, *process = parameters
        incidence = model.func(gen_time_mean, gen_time_sd, process, np.log(seed)).incidence
        return 0.0 - sum([(incidence[t] * cdr - d) ** 2 for t, d in targets.items()])

    def obj_func_ll(gen_time_mean, gen_time_sd, cdr_param, seed, proc_params):
        return calib_func_ll([gen_time_mean, gen_time_sd, cdr_param, seed] + list(proc_params), **calib_kwargs)

    return obj_func_ll

In [ ]:
priors = [esp.UniformPrior(k, (v['lower'], v['upper'])) for k, v in params_df.iterrows()]
priors.append(esp.UniformPrior(proc_req['name'], (proc_req['lower'], proc_req['upper']), size=n_process_periods))
renewal_model = RenewalModel(calib_kwargs['pop'], n_times, run_in, n_process_periods)
obj_func = get_obj_func(renewal_model)
n_draws = 100

with pm.Model() as pmm:
    variables = use_model(priors, obj_func)
    idata = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=n_draws)

In [ ]:
az.summary(idata)

In [ ]:
mean_posterior_params = mpp = np.array(az.summary(idata)['mean'])

In [ ]:
incidence = renewal_model.func(mpp[0], mpp[1], mpp[4:], mpp[3]).incidence * mpp[2]

In [ ]:
burn_in = 10
n_samples = 10
idata_burnt = idata.sel(draw=slice(10, None))
idata_sampled = az.extract(idata_burnt, num_samples=n_samples)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
spaghetti = pd.DataFrame()
for i, p in enumerate(sample_params):
    incidence = renewal_model.func(p['gen_mean'], p['gen_sd'], p['random_process'], np.log(p['seed'])).incidence
    cdr = p['cdr']
    spaghetti[i] = incidence * cdr
spaghetti.columns = sample_params.index.to_flat_index().map(str)

In [ ]:
# To get markdown-compatible dataframe for parameters
params_table = params_df.drop(columns=['name'])
params_table.index = params_df['name']
params_table = params_table.rename_axis(None)
params_table = params_table.rename(columns={'init': 'Starting value', 'lower': 'Lower limit', 'upper': 'Upper limit'})

In [ ]:
spaghetti['targets'] = calib_kwargs['targets']
spaghetti.plot()